# Tutorial 8: Cuantificación de LLM y Efficent Fine-tunning.

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes


### Objetivos del Tutorial

- Entender cuales son los problemas de almacenamiento de los LLM.
- Comprender que es la cuantificación de modelos.
- Como cargar un LLM cuantizado usando la librería de `bitandbytes`.
- Entender que es fine-tunning y su importancia. Conocer uno de lo algoritmo que ha optimizado este proceso.
- Revisar como realizar fine-tunning usando LoRA en BERT con la librería `peft`.

Los Large Language Models (LLMs) son conocidos por sus extensos requisitos computacionales. Típicamente, el tamaño de un modelo se calcula multiplicando el número de parámetros (tamaño) por la precisión de estos valores (tipo de dato). Sin embargo, para ahorrar memoria, los pesos pueden ser almacenados utilizando tipos de datos de menor precisión a través de un proceso conocido como cuantificación.

En la literatura se distinguen dos principales familias de técnicas de cuantificación de pesos:

- La Cuantificación Post-Entrenamiento (Post-Training Quantization, PTQ) es una técnica directa donde los pesos de un modelo ya entrenado se convierten a una precisión menor sin necesitar ningún reentrenamiento. Aunque es fácil de implementar, la PTQ se asocia con una posible degradación del rendimiento.
- El Entrenamiento Consciente de la Cuantificación (Quantization-Aware Training, QAT) incorpora el proceso de conversión de pesos durante la etapa de pre-entrenamiento o ajuste fino, lo que resulta en un rendimiento mejorado del modelo. Sin embargo, el QAT es computacionalmente costoso y exige datos de entrenamiento representativos.

Para trabajar con los LLMs cuantizados existe la librería `bitandbytes`, es una extensión para PyTorch que proporciona implementaciones eficientes de optimizadores de 4 y 8 bits. Su principal utilidad es mejorar la eficiencia del entrenamiento de modelos de deep learning, especialmente modelos grandes como los LLMs. Al usar bitsandbytes, se puede reducir el uso de memoria y acelerar las operaciones de entrenamiento sin una pérdida significativa de precisión. Esto es particularmente útil para entrenar modelos grandes en hardware con recursos limitados o para mejorar la velocidad de entrenamiento en hardware existente.

# Instalar librerías

En este tutorial, nos enfocaremos en utilizar optimizadores de 4 bits, utilizando la librerías `transformers` y `bitandbytes`

Primero necesitamos instalar las librerías necesarias:



In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.0/515.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Un ejemplo básico seria:

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Primero, necesitas entender los diferentes argumentos que pueden ser ajustados y utilizados.

Todos estos parámetros pueden ser cambiados utilizando BitsandBytesConfig de transformers y pasándolo al argumento quantization_config al llamar a from_pretrained.

¡Hay que asegurarse de pasar load_in_4bit=True cuando uses BitsAndBytesConfig!

#### Cambio de dtype

El dtype se utiliza para cambiar el tipo de datos que se utilizará durante la computación. Por ejemplo, los estados ocultos podrían estar en float32, pero el cálculo puede configurarse en bf16 para acelerar el proceso.

Por defecto, el dtype de cálculo está configurado en float32.

In [15]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [16]:
model_cd_bf16 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [25]:
print(model_cd_bf16)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e

In [21]:
text = "Hello my name is"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_cd_bf16.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello my name is Khaled and I'm 30 years old. My dad was a dancer, but he died


In [22]:
mb = model_cd_bf16.get_memory_footprint() / 1024**2
print(f"Footprint total según HF: {mb:.1f} MB")


Footprint total según HF: 430.4 MB


In [23]:
model_not_qnt = AutoModelForCausalLM.from_pretrained(model_id)

In [24]:
mb = model_not_qnt.get_memory_footprint() / 1024**2
print(f"Footprint total según HF: {mb:.1f} MB")

Footprint total según HF: 1884.6 MB


#### Cambiar del tipo de cuantización

La integración de 4 bits viene con dos tipos diferentes de cuantización: FP4 y NF4. El tipo de datos NF4 significa Normal Float 4 y se introduce en el [artículo de QLoRA](https://arxiv.org/abs/2305.14314).

Puedes cambiar entre estos dos tipos de datos utilizando bnb_4bit_quant_type de BitsAndBytesConfig. Por defecto, se utiliza la cuantización FP4.

In [26]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

In [27]:
outputs = model_nf4.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello my name is Kiki and I am a student at the University of Toronto. I have always been fascinated by the


In [33]:
mb = model_nf4.get_memory_footprint() / 1024**2
print(f"Footprint total según HF: {mb:.1f} MB")

Footprint total según HF: 430.4 MB


#### Uso de cuantización anidada para una inferencia y un entrenamiento más eficientes en memoria

Se aconseja utilizar la técnica de cuantización anidada. Esto ahorra más memoria sin pérdida de rendimiento adicional - según observaciones empíricas, esto permite el ajuste fino del modelo llama-13b en una NVIDIA-T4 de 16GB con una longitud de secuencia de 1024, tamaño de lote de 1 y pasos de acumulación de gradiente de 4.

Para habilitar esta característica, simplemente añade bnb_4bit_use_double_quant=True al crear tu configuración de cuantización.

In [28]:
from transformers import BitsAndBytesConfig

double_quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
)

model_double_quant = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=double_quant_config)

In [29]:
outputs = model_double_quant.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello my name is Yael and I'm a computer programmer. So what kind of programming language are you? 

I


#### Poniendo al límite las capacidades de Google Colab


¿Hasta dónde podemos llegar con la cuantización? Veremos a continuación que es posible cargar un modelo a escala de 20B (40 GB en precisión media) completamente en la GPU utilizando este método de cuantización. 🤯

Carguemos el modelo con el tipo de cuantización NF4 para obtener mejores resultados, dtype de cálculo bfloat16, así como cuantización anidada para una carga de modelo más eficiente en memoria.

In [30]:
"""
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
"""

'\nimport torch\nfrom transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig\n\nmodel_id = "EleutherAI/gpt-neox-20b"\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=torch.bfloat16\n)\n\ntokenizer = AutoTokenizer.from_pretrained(model_id)\nmodel_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")\n'

## ¿Qué es el Fine-tunning?

El Fine-tunning es una técnina ampliamente utilizada que permite la personalización de modelos de lenguaje preentrenados para tareas específicas. Este proceso requiere un conjunto de datos etiquetados para la completación de tareas, permitiendo que el modelo adapte sus pesos para alinearse de manera más efectiva con la tarea objetivo. Sin embargo, ajustar un modelo de lenguaje a gran escala desde cero puede ser un desafío en términos de los requisitos de GPU y el tiempo extenso que toma completar el entrenamiento. Es notable que hay LLM con hasta 7 mil millones de parámetros e incluso 1.76 billones de parámetros, como se ve en el caso de GPT-4. Por lo tanto, ajustar un LLM desde cero no siempre es la opción más eficiente.

### Como funciona LoRa

LORA surgió como solución cuando nos dimos cuenta de que el ajuste fino de un modelo de LLM desde cero requiere significativamente más parámetros que cuando se hace a partir de un modelo preentrenado. LORA ha demostrado que, para ajustar un modelo preentrenado, no es necesario modificar cada peso en cada capa. En cambio, introduce un método para aprender una representación de los pesos de la capa específica para la tarea en una dimensión más baja.

Desglosemos LORA paso a paso. Consideremos una capa completamente conectada con $m$ unidades de entrada y $n$ unidades de salida. La matriz de pesos para esta capa tiene dimensiones $m \times n$. Cuando proporcionamos una entrada $x$, la salida de esta capa se calcula utilizando la fórmula $Y = W X$.

Durante el fine-tunning con LORA, mantenemos $W$ fija e introducimos dos matrices, $A$ y $B$, en la ecuación. La nueva ecuación se convierte en $Y = W X + A \cdot B X$. Ahora, imagina que $m$ es 800 y $n$ es 3200, lo que da a la forma de $W$ 800 x 3200, resultando en 2.560.000 pesos.

Aquí es donde entra en juego la innovación de LORA. La matriz A tiene una forma de 800 x $r$, y la matriz B tiene una forma de $r$ x 3200. La clave es que puedes ajustar el valor de $r$. Si establecemos $r$ en 1, el número de pesos en esta capa se convierte en:

(800 x 1) + (1 x 3200) = 4000.

Esta es una reducción significativa en comparación con el ajuste fino con 2.560.000 pesos. En consecuencia, el fine-tunning de un LLM se vuelve mucho más rápido y requiere considerablemente menos recursos computacional, gracias al enfoque de LORA.

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*d1ckUy_f3nfdTP_J0xzs-g.png">

Ahora pasemos al código y expliquemos cómo ajustar BERT para la clasificación de texto usando LoRa.

### Instalar librerías

In [35]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


### Descargar el dataset

Comenzamos obteniendo el conjunto de datos IMDB con dos categorías de Hugging Face. Este conjunto de datos está diseñado para la clasificación binaria de sentimientos y cuenta con un volumen de datos significativamente mayor en comparación con los conjuntos de datos de referencia anteriores. Dentro de este conjunto, disponemos de 25,000 reseñas de películas fuertemente polarizadas para el entrenamiento, junto con otras 25,000 para pruebas. Además, existe un conjunto adicional de datos no etiquetados que se pueden utilizar según sea necesario.

In [36]:
from datasets import load_dataset

dataset = load_dataset("imdb")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [37]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### Preprocesamiento

Se requiere para el procesamiento de texto requiere el uso de un tokenizador, junto con la implementación de una estrategia de relleno y truncamiento para gestionar de manera efectiva las longitudes de secuencias variables.


In [38]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Podemos crear un conjunto de datos más pequeño para el ajuste fino con el fin de ahorrar tiempo y simplificar el proceso.

In [39]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [62]:
print(small_train_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1, 'input_ids': [101, 1247, 1110, 1185, 6796, 1120, 1155, 1206, 3144, 2852, 1105, 26890, 1197, 1133, 1103, 1864, 1115, 1241, 1132, 2021, 1326, 1164, 5973, 6969, 119, 26890, 1197, 2736, 19501, 1183, 117, 3144, 2852, 273

### Inyectar LoRA al modelo BERT

A continuación, establecemos un objeto de configuración LORA utilizando los parámetros de PEFT de Hugging Face. El parámetro `task_Type` especifica el tipo de tarea para la cual el modelo será ajustado finamente. El parámetro $r$ denota las dimensiones de $A$ y $B$, como se mencionó anteriormente. Además, el `lora_alpha` actúa como un factor de escala, determinando la importancia relativa de los pesos en $A$ y $B$ en relación con los parámetros originales del modelo.

Algunos ejemplos de task_Type:

- SEQ_CLS: una clase por texto (usa la representación de la secuencia, típicamente [CLS] en BERT-like o un pooled output).

- TOKEN_CLS: una clase por token (NER, POS, etc.).

- CAUSAL_LM: lenguaje autoregresivo (decoder-only) para generación; no es clasificación “con head” dedicada.

- SEQ_2_SEQ_LM: tareas encoder-decoder (traducción, resumen generativo).

In [40]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [41]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Posteriormente, insertamos las matrices $A$ y $B$ en nuestro modelo invocando la función `get_peft_model`.

In [42]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

### Entrenamiento y evaluación del modelo

El Trainer no realiza de manera inherente una evaluación automática del rendimiento del modelo durante el proceso de entrenamiento. Para evaluar el modelo, tendremos que proporcionar al Trainer una función personalizada para calcular e informar métricas. Podemos utilizar la función 'evaluate.load' de la biblioteca Evaluate de Hugging Face, que ofrece una función de precisión sencilla para este propósito

In [43]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Luego llamamos a la función compute en la métrica para calcular la precisión de las predicciones del modelo. Antes de pasar las predicciones a compute, necesitamos convertir las predicciones en logit.

In [64]:
def compute_metrics(eval_pred):
    logits, label = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=label)

A continuación, creamos una clase `TrainingArguments` que consolida un conjunto integral de hiperparámetros disponibles para personalización, junto con opciones de activación para diversas configuraciones de entrenamiento.

In [65]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", report_to="none" ,
                                 num_train_epochs=3)

In [66]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Finalmente entrenamos el modelo y observamos su rendimiento en cada época.

In [67]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=0.66187109375, metrics={'train_runtime': 221.1674, 'train_samples_per_second': 13.564, 'train_steps_per_second': 1.696, 'total_flos': 789687078912000.0, 'train_loss': 0.66187109375, 'epoch': 3.0})

In [77]:
# Ejemplo de texto para clasificación
text = "A stunningly well-made film, it was amazing"

# Tokenizar el texto
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [78]:
inputs.to('cuda:0')
outputs = model(**inputs)

# Obtener las probabilidades y la clase predicha
logits = outputs.logits
predicted_class = logits.argmax(dim=-1).item()

# Mostrar la clase predicha
print(f"Clase predicha: {predicted_class}")

Clase predicha: 0


In [79]:
# Ejemplo de texto para clasificación
text = "This movie was the worst movie I've ever seen"

# Tokenizar el texto
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

inputs.to('cuda:0')
outputs = model(**inputs)

# Obtener las probabilidades y la clase predicha
logits = outputs.logits
predicted_class = logits.argmax(dim=-1).item()

# Mostrar la clase predicha
print(f"Clase predicha: {predicted_class}")

Clase predicha: 0


In [55]:
model.save_pretrained("./lora_bert")
tokenizer.save_pretrained("./lora_bert")

('./lora_bert/tokenizer_config.json',
 './lora_bert/special_tokens_map.json',
 './lora_bert/vocab.txt',
 './lora_bert/added_tokens.json',
 './lora_bert/tokenizer.json')